# 函数调用(Function Calling)

文心一言提供函数调用功能，模型根据用户需求以及对函数的描述确定何时以及如何调用函数。函数调用功能的典型使用流程如下：

(1) 用户提供对一组函数的名称、功能、请求参数（输入参数）和响应参数（返回值）的描述； 

(2) 模型根据用户需求以及函数描述信息，智能确定是否应该调用函数、调用哪一个函数、以及在调用该函数时需要如何设置输入参数； 

(3) 用户根据模型的提示调用函数，并将函数的响应传递给模型； 

(4) 模型综合对话上下文信息，以自然语言形式给出满足用户需求的回答。

借由函数调用，用户可以从大模型获取结构化数据，进而利用编程手段将大模型与已有的内外部API结合以构建应用。

在ERNIE Bot SDK中，erniebot.ChatCompletion.create接口提供函数调用功能。关于该接口的更多详情请参考ChatCompletion API文档。

| 字段名称    | 数据类型 | 字段描述                         |
| ------------- | ---------- | ---------------------------------- |
| name        | str      | API（函数）名称                  |
| description | str      | API(函数）功能描述               |
| parameters  | dict     | API(函数）输入参数（以dict输入） |
| responses   | dict     | API(函数）输出结果（以dict返回） |

parameter 和 responses 的相关字段情况如下

| 字段名     | 数据类型  | 描述                                                |
| ------------ | ----------- | ----------------------------------------------------- |
| type       | str       | 此层级一般为object                                  |
| properties | dict      | 变量的详细情况，键为变量名，值为变量的相关信息（dict），该字典详见下表                 |
| required   | List[str] | 函数的输入参数中必须含有的变量，responses不含此字段 |

对于properties中的变量的字典，相关的键值对如下

| properties中的变量字典的元素 | 类型     | 描述                 |
| :----------------------------- | ---------- | ---------------------- |
| type                         | Required | 此层级一般为string，请注意函数内部转换     |
| description                  | Option   | 该变量的描述         |
| enum                         | Option   | 该变量的可选参数 |
| default                         | Option   | 该变量的默认参数 |

以下是一个简单的使用示例


In [66]:
from typing import List
class function_solver:
    def __init__(self):
        self.functions = []

    def add(
        self,
        name: str,
        description:str,
        parameters:dict,
        responses:dict
    ):
        """
        name[str]: API（函数）名称 
        description[str]: API（函数）功能描述 
        parameters[dict]: API（函数）输入参数，以字典的形式输入，参数名称为key，value必须含有type，properties
        responses[dict]: API（函数）输出结果，以字典的形式输出，参数名称为key，value必须含有type，properties
        """
        if 'type' not in parameters: 
            raise ValueError(f"Response 'parameters' must include 'type' field.")
        if 'type' not in responses:
            raise ValueError(f"Response 'responses' must include 'type' field.")

        self.functions.append({
            'name': name,
            'description': description,
            'parameters': parameters,
            'responses': responses
        })
    
    def fit(self) -> List[dict]:
        return self.functions
def add_function(a:str,b:str):
    return float(a) + float(b)

functions = function_solver()

functions.add(
    name = 'add_function',
    description = "计算两个数的和",
    parameters = {
                'type': 'object',
                'properties': {
                    'a': {
                        'type': 'string',
                        'description': "第一个数",
                    },
                    'b': {
                        'type': 'string',
                        'description': "第二个数",
                    },
                },
                'required': [
                    'a',
                    'b',
                ],
            },
    responses = {
                'type': 'object',
                'properties': {},
            }, 
)

def minus_function(a:str,b:str = '1.0'):
    return float(a) - float(b)

functions.add(
    name = 'minus_function',
    description = "计算两个数的差",
    parameters = {
                'type': 'object',
                'properties': {
                    'a': {
                        'type': 'string',
                        'description':'第一个数'
                    },
                    'b': {
                        'type': 'string',
                        'default':'1.0'
                    },
                },
                'required': [
                    'a',
                ],
            },
    responses = {
                'type': 'object',
                'properties': {},
            }, 
)

functions.fit()

[{'name': 'add_function',
  'description': '计算两个数的和',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'string', 'description': '第一个数'},
    'b': {'type': 'string', 'description': '第二个数'}},
   'required': ['a', 'b']},
  'responses': {'type': 'object', 'properties': {}}},
 {'name': 'minus_function',
  'description': '计算两个数的差',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'string', 'description': '第一个数'},
    'b': {'type': 'string', 'default': '1.0'}},
   'required': ['a']},
  'responses': {'type': 'object', 'properties': {}}}]

在有了添加完function之后，即可调用function calling功能让大模型进行思考以及决策

In [67]:
import erniebot,os
os.environ['EB_ACCESS_TOKEN']='d86186382de8cceb4512efbd774b74ea72f3a9f5'
erniebot.api_type = 'aistudio'
erniebot.access_token = os.getenv("EB_ACCESS_TOKEN")

response = erniebot.ChatCompletion.create(
    model='ernie-bot',
    messages=[{
        'role': 'user',
        'content': "1加2等于多少",
    }, ],
    functions = functions.fit(),
)
print(response.get_result())

{'name': 'add_function', 'thoughts': '用户的问题是关于1加2的计算，我可以使用add_function工具来计算这两个数的和。', 'arguments': '{"a":"1","b":"2"}'}


最后可以通过以下函数进行执行，获取最终想要的结果

In [68]:
eval(response.get_result()['name'])(**eval(response.get_result()['arguments']))

3.0

目前function call有点问题 
如1+2不行 必须要1加2  
北京今天天气怎么样不行，北京今天气温怎么样才可以